In [7]:
import torch
import numpy as np

from common.utils import save_reconstructions
from pathlib import Path
from collections import defaultdict
import fastmri
import h5py
import os

In [17]:
PUBLIC_ACCS = ["acc4", "acc5", "acc8"]
root_path = "/home/Data/leaderboard"
out_path = "reconstructions/grappa"

In [11]:
os.listdir(root_path)

['acc5', 'acc9']

In [ ]:
from dataset import SliceDataModule
import tqdm
dm = SliceDataModule(root="/home/Data")
dm.setup("test")
dm.setup("predict")
for (mask, kspace, target, maximum, fnames, slices) in tqdm.tqdm(dm.test_dataloader()):
    print(fnames, slices)
    print(type(slices))

In [18]:
for acc in os.listdir(root_path):
    reconstructions = defaultdict(dict)
    file_list = os.listdir(f"{root_path}/{acc}/image")
    for file in file_list:
        grappa = h5py.File(f"{root_path}/{acc}/image/{file}", "r")["image_grappa"]
        for i in range(grappa.shape[0]):
            reconstructions[file][i] = np.array(grappa[i])
    for fname in reconstructions:
        reconstructions[fname] = np.stack([reconstructions[fname][slice] for slice in sorted(reconstructions[fname])])

    if acc in PUBLIC_ACCS:
        save_reconstructions(reconstructions, Path(f"{out_path}/public"))

    else:
        save_reconstructions(reconstructions, Path(f"{out_path}/private"))